In [222]:
import pandas as pd
import openpyxl
import time

In [223]:
path_file = 'data/СВОД_4 кв_2023.xlsx'

In [224]:
# получаем список листов
temp_wb = openpyxl.load_workbook(path_file,read_only=True)
lst_file = temp_wb.sheetnames
temp_wb.close()

In [225]:
lst_file

['Свод_2019', 'Свод_2020', 'Свод_2021', 'Свод_2022', 'свод_2023']

In [226]:
# создаем датафрейм общий
all_df = pd.DataFrame()
# создаем словарь для хранения подсчитанных данных
itog_dct = {'Численность граждан Российской Федерации, обратившихся в ЦОПП для консультирования по построению индивидуальной траектории обучения – всего,':0,
            'Количество разработанных ЦОПП образовательных программ – всего,':0,
            'Численность граждан Российской Федерации, прошедших обучение по всем видам образовательных программ, предлагаемых ЦОПП – всего,':0,
            'Численность граждан Российской Федерации, прошедших обучение по программам, предлагаемым ЦОПП, трудоустроившихся в течение первого года с момента окончания обучения':0,
            'Численность граждан Российской Федерации, принявших участие в профориентационных мероприятиях, проводимых ЦОПП, в том числе профессиональных пробах':0,
            '(из п.14) Численность обучающихся в 6 – 11 классах общеобразовательных организаций, принявших участие в профориентационных мероприятиях ЦОПП':0,
            'Численность привлеченных ЦОПП сотрудников (работников) научных организаций и образовательных организаций, реализующих программы среднего профессионального, высшего образования и дополнительного профессионального образования, представителей промышленных предприятий и высокотехнологичного бизнеса, представителей иных организаций для реализации мероприятий по профессиональной ориентации, в том числе для учащихся общеобразовательных организаций':0,
            '– программам обучения в рамках проекта «Первая профессия»':0,
            'Количество организаций–партнеров, с которыми налажено взаимодействие ЦОПП по вопросам опережающей профессиональной подготовки на основе договоров/соглашений':0,
            'Информационное сопровождение деятельности ЦОПП на региональном уровне':0,
            'Информационное сопровождение деятельности ЦОПП на федеральном уровне':0}

In [227]:
for name_sheet in lst_file:
    print(name_sheet)
    temp_reg_df = pd.read_excel(path_file,sheet_name=name_sheet,nrows=1)
    # Получаем регионы на листе
    temp_lst_regions = [region for region in temp_reg_df.columns if 'Unnamed' not in region]
    # начинаем обработку непосредственно данных
    temp_df = pd.read_excel(path_file,sheet_name=name_sheet,skiprows=1)

    # удаляем лишние колонки
    temp_df.drop(columns=['№ показателя','№ строки','Единица измерения'],inplace=True)
   

    # убираем лишние строки с значением 1 или 2
    temp_df = temp_df[~temp_df['Наименование индикатора/показателя'].isin([1,2])]

    temp_reg_df = temp_df.copy() # получаем копию датафрейма чтобы получить добавить названия регионов

    # оставляем только колонки за 2023 т.е те что посередине
    lst_2023_columns = [column
                        for column in temp_reg_df.columns
                        if 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)'
                       in column
                     ] 
    
    
        # оставляем только колонки с 2023
    temp_reg_df = temp_reg_df[lst_2023_columns]

    # заменяем названия колонок на регионы
    temp_reg_df.columns = temp_lst_regions

    temp_reg_df.insert(0,'Наименование индикатора/показателя',temp_df['Наименование индикатора/показателя'])

    # очищаем от пустых строк
    temp_reg_df.dropna(how='all',inplace=True)

    transpose_df=temp_reg_df.set_index('Наименование индикатора/показателя').transpose()
    
    transpose_df.fillna(0,inplace=True) # заполняем нули
    # Добавляем колонку для проверки
    transpose_df.insert(0,'Год cоздания',name_sheet)
    
    # начинаем подсчет
    for name_column in itog_dct.keys():
        print(name_column)
        transpose_df[name_column] = pd.to_numeric(transpose_df[name_column],errors='coerce')
        transpose_df[name_column].fillna(0,inplace=True)
        itog_dct[name_column] +=transpose_df[name_column].sum(numeric_only=True)
    
    # добавляем в общий датафрейм для проверки
    all_df = pd.concat([all_df,transpose_df])
    

Свод_2019
Численность граждан Российской Федерации, обратившихся в ЦОПП для консультирования по построению индивидуальной траектории обучения – всего,
Количество разработанных ЦОПП образовательных программ – всего,
Численность граждан Российской Федерации, прошедших обучение по всем видам образовательных программ, предлагаемых ЦОПП – всего,
Численность граждан Российской Федерации, прошедших обучение по программам, предлагаемым ЦОПП, трудоустроившихся в течение первого года с момента окончания обучения
Численность граждан Российской Федерации, принявших участие в профориентационных мероприятиях, проводимых ЦОПП, в том числе профессиональных пробах
(из п.14) Численность обучающихся в 6 – 11 классах общеобразовательных организаций, принявших участие в профориентационных мероприятиях ЦОПП
Численность привлеченных ЦОПП сотрудников (работников) научных организаций и образовательных организаций, реализующих программы среднего профессионального, высшего образования и дополнительного профессио

In [247]:
itog_dct

{'Численность граждан Российской Федерации, обратившихся в ЦОПП для консультирования по построению индивидуальной траектории обучения – всего,': 153101,
 'Количество разработанных ЦОПП образовательных программ – всего,': 6029,
 'Численность граждан Российской Федерации, прошедших обучение по всем видам образовательных программ, предлагаемых ЦОПП – всего,': 287160,
 'Численность граждан Российской Федерации, прошедших обучение по программам, предлагаемым ЦОПП, трудоустроившихся в течение первого года с момента окончания обучения': 43402,
 'Численность граждан Российской Федерации, принявших участие в профориентационных мероприятиях, проводимых ЦОПП, в том числе профессиональных пробах': 1460769,
 '(из п.14) Численность обучающихся в 6 – 11 классах общеобразовательных организаций, принявших участие в профориентационных мероприятиях ЦОПП': 1015371,
 'Численность привлеченных ЦОПП сотрудников (работников) научных организаций и образовательных организаций, реализующих программы среднего про

In [250]:
out_df=pd.DataFrame.from_dict(itog_dct,orient='index').rename(columns={0:'Итого'})

In [252]:
out_df['Итого'] = out_df['Итого'].astype(int)

In [256]:
all_df

Наименование индикатора/показателя,Год cоздания,Созданы цифровая платформа и сайт ЦОПП в сети Интернет,"Количество обращений, зарегистрированных на цифровой платформе ЦОПП \n(не может быть = охвату)","Численность граждан Российской Федерации, охваченных деятельностью ЦОПП всего,",из них (из п.3) в возрасте:\n– до 16 лет,– от 16 до 21 года,– от 22 до 35 лет,– от 36 до 55 лет,– от 56 до 75 лет,– старше 75 лет,...,Количество штатных единиц согласно утвержденному штатному расписанию ЦОПП,"Количество фактически работающих человек – всего,","из них (33):\n– количество сотрудников, работающих по договору ГПХ","Доля сотрудников ЦОПП, прошедших повышение квалификации","Количество предприятий ОПК, с которыми налажено взаимодействие","(из п. 10) Численность граждан, прошедших обучение по запросу предприятий ОПК","(из п.11) Количество программ (или проф. модулей), разработанных под заказ предприятий ОПК","(из п. 3) Численность граждан с ОВЗ и инвалидов, обратившихся в ЦОПП","Наличие брендовой сувенирной продукции ЦОПП (раздаточный материал: буклет, брошюра, флаер и т.п.)",Наличие брендовой сувенирной продукции ЦОПП (другое)
Амурская область,Свод_2019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Астраханская область,Свод_2019,https://copp.edu.astrobl.ru/,252,20046,13943,3798,1640,590,73,2,...,Х,Х,Х,100,1,0,0,5,Х,Х
Волгоградская область,Свод_2019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Воронежская область,Свод_2019,copp36.ru,1454,28276,17370,7442,1121,1984,359,0,...,Х,Х,Х,38,22,0,0,116,Х,Х
Иркутская область,Свод_2019,"https://cp.copp38.ru , https://copp38.ru /",6,16606,5906,7813,1947,826,112,2,...,26,Х,Х,Х,43,0,0,548,X,Х
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Удмуртская Республика,свод_2023,https://copp18.ru/,1826,5340,2332,2069,260,572,106,1,...,Х,Х,Х,26,7,0,0,3,Х,Х
Ульяновская область,свод_2023,https://copp73.ru/ https://online.copp73.ru/,3816,12877,7167,5496,72,124,18,0,...,Х,Х,Х,0.385,6,13,6,419,Х,Х
Чеченская Республика,свод_2023,http://copp95.ru,822,4449,3151,670,305,240,82,0,...,Х,Х,Х,96,0,0,0,0,Х,Х
Чувашская Республика,свод_2023,online.copp21.ru\nhttps://copp21.ru/,286,18376,15992,1017,502,778,87,0,...,Х,Х,Х,33.3,49,199,5,7,Х,Х


In [258]:
all_df.drop(columns=['Созданы цифровая платформа и сайт ЦОПП в сети Интернет'],inplace=True)

In [259]:
all_df.to_excel('data/Проверка.xlsx',header=True,index=True)

In [254]:
out_df.to_excel('data/Данные для отчета.xlsx',header=True,index=True)

In [228]:
temp_reg_df = pd.read_excel(path_file,sheet_name='Свод_2021',nrows=1)

In [229]:
# Получаем регионы на листе
temp_lst_regions = [region for region in temp_reg_df.columns if 'Unnamed' not in region]

In [230]:
temp_df = pd.read_excel(path_file,sheet_name='Свод_2021',skiprows=1)

In [231]:
# удаляем лишние колонки
temp_df.drop(columns=['№ показателя','№ строки','Единица измерения'],inplace=True)

In [232]:
# убираем лишние строки с значением 1 или 2
temp_df = temp_df[~temp_df['Наименование индикатора/показателя'].isin([1,2])]

In [233]:
temp_reg_df = temp_df.copy() # получаем копию датафрейма чтобы получить регионы

In [234]:
# оставляем только колонки за 2023 т.е те что посередине
lst_2023_columns = [column
                    for column in temp_reg_df.columns
                    if 'Фактическое значение по итогам четырёх кварталов 2023 г. (указывается значение суммы I, II, III, IV кварталов)'
                   in column
                 ] 

In [235]:
# оставляем только колонки с 2023
temp_reg_df = temp_reg_df[lst_2023_columns]

In [236]:
# заменяем названия колонок на регионы
temp_reg_df.columns = temp_lst_regions

In [237]:
temp_reg_df.insert(0,'Наименование индикатора/показателя',temp_df['Наименование индикатора/показателя'])

In [238]:
# очищаем от пустых строк
temp_reg_df.dropna(how='all',inplace=True)

In [239]:
transpose_df=temp_reg_df.set_index('Наименование индикатора/показателя').transpose()

In [240]:
transpose_df.head()

Наименование индикатора/показателя,Созданы цифровая платформа и сайт ЦОПП в сети Интернет,"Количество обращений, зарегистрированных на цифровой платформе ЦОПП \n(не может быть = охвату)","Численность граждан Российской Федерации, охваченных деятельностью ЦОПП всего,",из них (из п.3) в возрасте:\n– до 16 лет,– от 16 до 21 года,– от 22 до 35 лет,– от 36 до 55 лет,– от 56 до 75 лет,– старше 75 лет,"(из п.3) Численность граждан в возрасте от 25 до 65 лет, принявших участие в мероприятиях и программах ЦОПП, в том числе по профессиональной ориентации, ускоренному профессиональному обучению, дополнительному профессиональному образованию, по перспективным профессиям и компетенциям",...,Количество штатных единиц согласно утвержденному штатному расписанию ЦОПП,"Количество фактически работающих человек – всего,","из них (33):\n– количество сотрудников, работающих по договору ГПХ","Доля сотрудников ЦОПП, прошедших повышение квалификации","Количество предприятий ОПК, с которыми налажено взаимодействие","(из п. 10) Численность граждан, прошедших обучение по запросу предприятий ОПК","(из п.11) Количество программ (или проф. модулей), разработанных под заказ предприятий ОПК","(из п. 3) Численность граждан с ОВЗ и инвалидов, обратившихся в ЦОПП","Наличие брендовой сувенирной продукции ЦОПП (раздаточный материал: буклет, брошюра, флаер и т.п.)",Наличие брендовой сувенирной продукции ЦОПП (другое)
Белгородская область,"https://xn--31-kmc.xn--n1acaz.xn--p1ai/ , http...",1442,127548,75749,25614,13034,12419,732,0,24910,...,43,38,13,100,23,361,71,532,Х,Х
Брянская область,https://copp32.ru/,732,25266,12989,7730,1266,2524,757,0,4059,...,Х,Х,Х,31,5,74,14,0,Х,Х
Владимирская область,https://xn--n1acaz.xn--33-6kcadhwnl3cfdx.xn--p...,4141,4970,2115,2108,548,180,19,0,517,...,Х,Х,Х,40,47,13,2,27,Х,Х
Вологодская область,https://online.copp35.ru/\nhttps://copp35.ru/,2,11011,5434,4750,306,418,103,0,341,...,Х,Х,Х,0.143,5,0,0,0,Х,Х
Кировская область,htpps://copp43.ru,37,19623,9210,8681,1127,504,101,0,1509,...,Х,Х,Х,0,34,299,13,687,Х,Х


In [241]:
transpose_df.fillna(0,inplace=True) # заполняем нули
# Добавляем колонку для проверки
transpose_df.insert(0,'Год cоздания',name_sheet)


In [242]:
print(transpose_df['Численность граждан Российской Федерации, обратившихся в ЦОПП для консультирования по построению индивидуальной траектории обучения – всего,'])

Белгородская область     40957
Брянская область             0
Владимирская область       728
Вологодская область       1609
Кировская область          288
Курская область             55
Пензенская область        2302
Рязанская область           22
Тамбовская область         415
Тульская область           473
Удмуртская Республика      192
Ульяновская область        220
Чеченская Республика        80
Чувашская Республика        25
Ярославская область        106
Name: Численность граждан Российской Федерации, обратившихся в ЦОПП для консультирования по построению индивидуальной траектории обучения – всего,, dtype: object


In [243]:
# начинаем подсчет
for name_column in itog_dct.keys():
    print(name_column)
    transpose_df[name_column] = pd.to_numeric(transpose_df[name_column],errors='coerce')
    
    itog_dct[name_column] +=transpose_df[name_column].sum(numeric_only=True)

Численность граждан Российской Федерации, обратившихся в ЦОПП для консультирования по построению индивидуальной траектории обучения – всего,
Количество разработанных ЦОПП образовательных программ – всего,
Численность граждан Российской Федерации, прошедших обучение по всем видам образовательных программ, предлагаемых ЦОПП – всего,
Численность граждан Российской Федерации, прошедших обучение по программам, предлагаемым ЦОПП, трудоустроившихся в течение первого года с момента окончания обучения
Численность граждан Российской Федерации, принявших участие в профориентационных мероприятиях, проводимых ЦОПП, в том числе профессиональных пробах
(из п.14) Численность обучающихся в 6 – 11 классах общеобразовательных организаций, принявших участие в профориентационных мероприятиях ЦОПП
Численность привлеченных ЦОПП сотрудников (работников) научных организаций и образовательных организаций, реализующих программы среднего профессионального, высшего образования и дополнительного профессионального о

In [244]:
itog_dct

{'Численность граждан Российской Федерации, обратившихся в ЦОПП для консультирования по построению индивидуальной траектории обучения – всего,': 153101,
 'Количество разработанных ЦОПП образовательных программ – всего,': 6029,
 'Численность граждан Российской Федерации, прошедших обучение по всем видам образовательных программ, предлагаемых ЦОПП – всего,': 287160,
 'Численность граждан Российской Федерации, прошедших обучение по программам, предлагаемым ЦОПП, трудоустроившихся в течение первого года с момента окончания обучения': 43402,
 'Численность граждан Российской Федерации, принявших участие в профориентационных мероприятиях, проводимых ЦОПП, в том числе профессиональных пробах': 1460769,
 '(из п.14) Численность обучающихся в 6 – 11 классах общеобразовательных организаций, принявших участие в профориентационных мероприятиях ЦОПП': 1015371,
 'Численность привлеченных ЦОПП сотрудников (работников) научных организаций и образовательных организаций, реализующих программы среднего про

In [245]:
all_df = pd.concat([all_df,transpose_df])